In [17]:

import numpy as np
import pandas as pd


In [19]:
import pandas as pd

# Correct way to specify file path and sheet name
file_path = "D:/Polymers/extracted/Ramp 5.00 °Cmin to 800.00 °C.xlsx" # This should be a string
sheet_name = 0  # This is passed separately

# Load the Excel file
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Optional: check the first few rows
print(df.head())


   Temperature cel  Weight%  Heat Flow (Normalized) W/g  Time min       1/RT  \
0              301   99.511                       0.166     11.96  69.058215   
1              302   99.503                       0.169     12.13  69.178494   
2              303   99.494                       0.170     12.31  69.298773   
3              304   99.485                       0.170     12.49  69.419052   
4              304   99.485                       0.170     12.49  69.419052   

   ln(B/T^2)  Efficiency   
0  -9.804783      0.00489  
1  -9.811416      0.00497  
2  -9.818028      0.00506  
3  -9.824617      0.00515  
4  -9.824617      0.00515  


In [33]:

# Ensure proper column names
temperature_col = 'Temperature cel'
weight_col = 'Weight%'
heat_flow= 'Heat Flow (Normalized) W/g'
Time_min='Time min'
X='1/RT'
Y='ln(B/T^2)'
conv_col='conversion'


df[conv_col] = (
    df[conv_col]                       # original column (any type)
      .astype(str)                    # make sure it’s a string
      .str.extract(r'([-+]?\d*\.?\d+)')  # first numeric substring
      .astype(float)                  # to float
      .round(2)                       # two decimals
)

# Extract only Temperature and Weight%
result = df[df[conv_col].notna()].copy()
result.head()

# result = filtered_df[[temperature_col, weight_col,heat_flow,Time_min,X,Y,Efficiency_col]]

,Temperature cel,Weight%,Heat Flow (Normalized) W/g,Time min,1/RT,ln(B/T^2),Efficiency
0,301,99.511,0.166,11.96,69.058215,-9.804783,0.00
1,302,99.503,0.169,12.13,69.178494,-9.811416,0.00
2,303,99.494,0.170,12.31,69.298773,-9.818028,0.01
3,304,99.485,0.170,12.49,69.419052,-9.824617,0.01
4,304,99.485,0.170,12.49,69.419052,-9.824617,0.01


In [35]:
# Optional: Save to new Excel
result.to_excel("D:/Polymers/extracted/efficiency_300_800_5ramp.xlsx", index=False)

In [37]:
import pandas as pd
from openpyxl import load_workbook

# --- 1. load ------------------------------------------------------
file_path   = r"D:/Polymers/extracted/efficiency_300_800_5ramp.xlsx"          # <‑‑ change this
sheet       = "Sheet1"                           # <‑‑ change this if needed
df = pd.read_excel(file_path, sheet_name=sheet)

# --- 2. drop rows that are complete duplicates -------------------
df = df.drop_duplicates()                # keep='first' is default

# --- 3. group (merge) on identical efficiency values -------------
# Build an aggregation rule for every column except 'efficiency'
agg_rules = {
    col: ("mean" if pd.api.types.is_numeric_dtype(df[col]) else "first")
    for col in df.columns if col != "conversion"
}

df_merged = (
    df
    .groupby("conversion", as_index=False)  # keep 'efficiency' as a normal column
    .agg(agg_rules)                         # apply the rules
)

# OPTIONAL: round any floats coming from the mean, e.g. to 2 dp
df_merged = df_merged.round(2)

In [39]:
print(df_merged)

    Efficiency   Temperature cel  Weight%  Heat Flow (Normalized) W/g  \
0          0.00           301.50    99.51                        0.17   
1          0.01           340.57    99.08                        0.27   
2          0.02           388.52    98.11                        0.40   
3          0.03           400.50    97.04                        0.39   
4          0.04           405.83    96.09                        0.38   
5          0.05           410.00    95.03                        0.36   
6          0.06           413.00    94.05                        0.34   
7          0.07           415.50    93.07                        0.31   
8          0.08           417.50    92.09                        0.27   
9          0.09           419.50    90.93                        0.23   
10         0.10           421.00    89.95                        0.21   
11         0.11           422.00    89.19                        0.20   
12         0.12           423.50    87.98          

In [45]:
# --- save df_merged to a new (second) sheet ------------------------
with pd.ExcelWriter(
        file_path,              # same workbook
        engine="openpyxl",
        mode="a",               # append, don’t overwrite the file
        if_sheet_exists="replace"   # replace if a sheet with this name already exists
    ) as writer:
    df_merged.to_excel(writer, sheet_name="Cleaned", index=False)


In [49]:
import pandas as pd
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt

# ------------------------------------------------------------
# 1. LOAD THE DATA (adjust the path/sheet as needed)
file_path  = r"D:/Polymers/extracted/efficiency_300_800_5ramp.xlsx"
sheet_name = "Cleaned"

df = pd.read_excel(file_path, sheet_name=sheet_name)
df.head()

,Efficiency,Temperature cel,Weight%,Heat Flow (Normalized) W/g,Time min,1/RT,ln(B/T^2)
0,0.00,301.50,99.51,0.17,12.05,69.12,-9.81
1,0.01,340.57,99.08,0.27,19.75,73.82,-10.05
2,0.02,388.52,98.11,0.40,29.25,79.59,-10.32
3,0.03,400.50,97.04,0.39,31.63,81.03,-10.38
4,0.04,405.83,96.09,0.38,32.70,81.67,-10.40


In [51]:
from scipy.stats import linregress

x = df['1/RT'].astype(float).to_numpy()
y = df['ln(B/T^2)'].astype(float).to_numpy()

slope, _, _, _, _ = linregress(x, y)
print(f"Slope: {slope:.6e}")


Slope: -3.668838e-02
